In [2]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import pandas as pd
import mne
data_dir = 'data_meg'
subj = "R2490"
dataqual = 'prepro' #or loc/exp
exp = 'loc' #or exp
dtype = "raw"
label_dir = 'data_log'
save_dir = 'data_meg'



In [3]:
# raw = mne.io.read_raw_fif('data_meg/R2490/prepro/R2490_exp.fif', preload='temp_raw.fif')
raw = mne.io.read_raw_fif(f'{data_dir}/{subj}/{dataqual}/{subj}_{exp}.fif', preload=True)
sfreq = raw.info['sfreq']
raw.filter(1, 40, method='iir')
downsample = 10
raw.resample(sfreq / downsample)

Opening raw data file data_meg/R2490/prepro/R2490_loc.fif...
    Range : 0 ... 669999 =      0.000 ...   669.999 secs
Ready.
Reading 0 ... 669999  =      0.000 ...   669.999 secs...


/var/folders/79/m99pb_8d67l3d79st4zhf4tc0000gn/T/ipykernel_19678/1237062969.py:2: RuntimeWarning: This filename (data_meg/R2490/prepro/R2490_loc.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(f'{data_dir}/{subj}/{dataqual}/{subj}_{exp}.fif', preload=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB

425 events found on stim channel STI 014
Event IDs: [160 164 165 166]
362 events found on stim channel STI 014
Event IDs: [160 164 165 166]


/var/folders/79/m99pb_8d67l3d79st4zhf4tc0000gn/T/ipykernel_19678/1237062969.py:6: RuntimeWarning: Resampling of the stim channels caused event information to become unreliable. Consider finding events on the original data and passing the event matrix as a parameter.
  raw.resample(sfreq / downsample)


<Raw | R2490_loc.fif, 193 x 67000 (670.0 s), ~100.2 MB, data loaded>

In [4]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 160,
    'move': 161,
    'reveal_red': 162,
    'reveal_white': 163,
    'done': 164,
    'choice': 165,
    'timeout': 166
}

# Define trials to remove
trials_to_remove = []

362 events found on stim channel STI 014
Event IDs: [160 164 165 166]


In [5]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
timeout_events = events[events[:, 2] == event_id['timeout']]
choice_events = events[events[:, 2] == event_id['choice']]
sfreq = raw.info['sfreq']

In [ ]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
choice_events = events[events[:, 2] == event_id['choice']]

sfreq = raw.info['sfreq']  # Sampling frequency


# Initialize a list to store trial information
trial_info = []
start_idx = 0

# Iterate through each start event to create trial information
for idx, start_event in enumerate(start_events):
    start_sample = start_event[0]

    # Find the next end event after the start event
    end_idx = np.searchsorted(done_events[:, 0], start_sample, side='right')
    if end_idx < len(done_events):
        end_sample = done_events[end_idx, 0]
        # Calculate tmin and tmax for the epoch
        tmin = -0.2  # 0.2 s before 'start'
        tmax = 26   # Duration from 'start' to 1 s after end event
        print(f"tmax is {tmax}")

        choice_event = choice_events[(choice_events[:, 0] > start_sample) & 
                                    (choice_events[:, 0] < end_sample)]
        choice_time = choice_event[0, 0]


        # Store trial information, including whether 'reveal_red' and 'reveal_white' occurred
        trial_info.append({
            'event_sample': start_sample,
            'trial_index': start_idx,
            'duration': 26,
            'tmin': tmin,
            'tmax': 26,
            'done': len(done_events) > 0,
            'start_times': start_sample / sfreq,
            'choice_event': len(choice_event) > 0,
            'choice_time': choice_time / sfreq if len(choice_event) > 0 else []
        })
        start_idx += 1